In [1]:
import os
import json
import re
from langchain import Anthropic
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage,
)
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.agents import tool
from pydantic import BaseModel, Field
from solcx import install_solc, compile_source
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from utils.tools import compile_solidity, SolidityCompilerInput



load_dotenv()


def get_secret(name):
    if os.environ[name]:
        return os.environ[name]
    return os.getenv(name)


Anthropic.anthropic_api_key = get_secret('ANTHROPIC_API_KEY')

with open('data/e2e_suggest_test_from_tests_example.json', 'r') as f_in:
    example = json.load(f_in)

In [2]:
print(f"example.keys():\n{example.keys()}\n")
print(f"example['assistant_inception_prompt'][:30]:\n{example['assistant_inception_prompt'][:30]}\n")
print(f"example['user_inception_prompt'][:30]:\n{example['user_inception_prompt'][:30]}\n")
print(f"example['complete_example'].keys():\n{example['complete_example'].keys()}\n")
print(f"example['incomplete_example'].keys():\n{example['incomplete_example'].keys()}\n")
print(f"example['user_task']:\n{example['user_task']}\n")

example.keys():
dict_keys(['assistant_inception_prompt', 'user_inception_prompt', 'incomplete_example', 'complete_example', 'complete_example_answer', 'user_task'])

example['assistant_inception_prompt'][:30]:
Never forget you are a {assist

example['user_inception_prompt'][:30]:
Never forget you are a {user_r

example['complete_example'].keys():
dict_keys(['contract_code', 'test_contract_code', 'function'])

example['incomplete_example'].keys():
dict_keys(['contract_code', 'test_contract_code', 'function'])

example['user_task']:

Here is the smart contract code to be tested: {contract_code}
Here is the test contract in Solidity: {test_contract_code}
Here is the specific function you need to test in the new end-to-end test: {function}
Recommend a new end-to-end test wrapped in a Solidity function with no other comments or explanation.



In [3]:
@tool(return_direct=False)
def compile_solidity(solidity_code_str):
    """
    This function compiles Solidity files using a specific version of solc.
    :return: Dict, the compiled contracts.
    """
    solidity_code_str = solidity_code_str.strip('`')
    solc_version = '0.8.2'
    # Install the specified version of solc
    install_solc(solc_version)
    try:
        result = compile_source(solidity_code_str)
    except Exception as e:
        print(f"couldn't compile {solidity_code_str}\n{str(e)}")
        return f'Compilation Failed. Either the code is invalid or current compiler({solc_version}) didnt match'
    print(result)
    return 'Compilation Successful. It is a valid solidity code.'


class SolidityCompilerInput(BaseModel):
    question: str = Field(description="Should be pure solidity code without surrounded backticks")

        
def _handle_error(error) -> str:
    return str(error)[:50]     
      
    
class CAMELAgent:

    def __init__(
            self,
            system_message: SystemMessage,
            model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        tools = [
            Tool(
                func=compile_solidity,
                name="solidity compiler",
                description="Can compile solidity code. Accepts ONLY solidity code",
                args_schema=SolidityCompilerInput,
            ),
        ]
        self.model = initialize_agent(
            tools, 
            model,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
            handle_parsing_errors=_handle_error)
#         self.model = model

        self.init_messages()
  


    def reset(self):
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(
            self,
            input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model(messages)
        self.update_messages(output_message)

        return output_message

def filter_content(names_bodies):
    for name, body in names_bodies:
        if len(name.split()) > 1:
            if 'function' in name and len(name.split('function')[-1].split()) == 1:
                name = name.split('function')[-1]
            else:
                continue
        before_open_paranthesis = body.split('(', 1)[0]
        if len(before_open_paranthesis.split()) > 2:
            if body.split('(', 1)[0].split()[-2] == 'function':
                name = before_open_paranthesis.split()[-1]
                body = f"function {name}({body.split('(', 1)[1]}"
            else:
                continue
        elif len(body.split('(', 1)) == 1:
            continue
        if not body.startswith('function'):
            print(f'body doesnt start with "function", body: {body}, name: {name}')

        if len(body.split('(', 1)[0].split()) != 2:
            print(f'text before opening parenthesis must be 2 words, body: {body}, name: {name}')
        yield name, body



def get_functions_dict(text):
    # Regex pattern to match function definitions
    pattern = r"(function\s+[^\(]+\([^\)]*\)\s*(public|external|internal|private)*\s*(?:pure|constant|view|payable)*\s*(?:returns)*\s*\(*[^\)]*\)*\s*\{[^}]+\})"
    matches = re.findall(pattern, text, re.MULTILINE)
    # Get a list of function names
    function_names = [re.search(r"(function\s+)([^\(]+)", match[0]).group(2).strip() for match in matches]
    # Get a list of function bodies
    function_bodies = [match[0] for match in matches]
    # Zip the function names and bodies into a dictionary
    names_bodies = zip(function_names, function_bodies)
    names_bodies = list(filter_content(names_bodies))
    functions_dict = dict(names_bodies)
    return functions_dict



In [4]:
class AgentsLlmApiCaller:

    def __init__(self, data):
        self.refresh = False
        self.user_role_name = "Solidity Developer"
        self.assistant_role_name = "Web3 Security Expert"
        self.variables = data['complete_example']
        self.example = data['complete_example_answer']
        self.assistant_inception_prompt = data['assistant_inception_prompt']
        self.user_inception_prompt = data['user_inception_prompt']
        self.user_task = data['user_task']

    def get_sys_msgs(self, assistant_role_name: str, user_role_name: str, task: str):
        assistant_sys_template = SystemMessagePromptTemplate.from_template(template=self.assistant_inception_prompt)
        assistant_sys_msg = \
            assistant_sys_template.format_messages(assistant_role_name=assistant_role_name,
                                                   user_role_name=user_role_name,
                                                   task=task)[0]

        user_sys_template = SystemMessagePromptTemplate.from_template(template=self.user_inception_prompt)
        user_sys_msg = \
            user_sys_template.format_messages(assistant_role_name=assistant_role_name,
                                              user_role_name=user_role_name,
                                              task=task)[0]

        return assistant_sys_msg, user_sys_msg

    def get_answer_query(self, contract_code, test_contract_code, function):
        user_task = self.user_task.format(
            contract_code=contract_code, test_contract_code=test_contract_code, function=function)
        word_limit = 10000  # word limit for task brainstorming
        task_specifier_prompt = (
            """Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
            Please make it more specific. Be creative and imaginative.
            Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
        )
        task_specifier_template = HumanMessagePromptTemplate.from_template(template=task_specifier_prompt)
        task_specifier_msg = task_specifier_template.format_messages(assistant_role_name=self.assistant_role_name,
                                                                     user_role_name=self.user_role_name,
                                                                     task=user_task, word_limit=word_limit)[0]
        assistant_sys_msg, user_sys_msg = self.get_sys_msgs(self.assistant_role_name, self.user_role_name,
                                                            task_specifier_msg.content)
        assistant_agent = CAMELAgent(assistant_sys_msg, ChatAnthropic(model="claude-v1-100k", temperature=0., max_tokens_to_sample=4096))
        user_agent = CAMELAgent(user_sys_msg, ChatAnthropic(model="claude-v1-100k", temperature=0., max_tokens_to_sample=4096))

        # Reset agents
        assistant_agent.reset()
        user_agent.reset()

        # Initialize chats
        assistant_msg = HumanMessage(
            content=(f"{user_sys_msg.content}. "
                     "Now start to give me introductions one by one. "
                     "Only reply with Instruction and Input."))

        user_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
        assistant_agent.step(user_msg)

        print(f"Original task prompt:\n{user_task}\n")

        result = {}
        chat_turn_limit = 8
        for n in range(chat_turn_limit):
            user_ai_msg = user_agent.step(assistant_msg)
            user_msg = HumanMessage(content=user_ai_msg.content)
            print(f"AI User ({self.user_role_name}):\n\n{user_msg.content}\n\n")
            assistant_ai_msg = assistant_agent.step(user_msg)
            assistant_msg = HumanMessage(content=assistant_ai_msg.content)
            print(f"AI Assistant ({self.assistant_role_name}):\n\n{assistant_msg.content}\n\n")
            if 'Solution: ' in assistant_msg.content:
                result.update(get_functions_dict(assistant_msg.content))
            if "<CAMEL_TASK_DONE>" in assistant_msg.content or "<CAMEL_TASK_DONE>" in user_msg.content:
                break
        return result


In [5]:
agent = AgentsLlmApiCaller(example)
result = agent.get_answer_query(**example['incomplete_example'])
print(f'\nfinal result is:\n{result}')
print(f'\nresult functions:\n{result.keys()}')

KeyboardInterrupt: 

In [ ]:
print(example['complete_example']['contract_code'][:1000])

In [ ]:
!solc-select use 0.8.17

In [6]:
example['complete_example']['contract_code'] = """pragma solidity ^0.8.17;

contract SimpleStorage {
    // State variable to store a number
    uint public num;

    // You need to send a transaction to write to a state variable.
    function set(uint _num) public {
        num = _num;
    }

    // You can read from a state variable without sending a transaction.
    function get() public view returns (uint) {
        return num;
    }
}"""
example['complete_example']['test_contract_code'] = ""
example['complete_example']['function'] = "set"
example['complete_example_answer'] = """pragma solidity ^0.8.17;

import "truffle/Assert.sol";
import "truffle/DeployedAddresses.sol";
import "../contracts/SimpleStorage.sol";

contract TestSimpleStorage {
    function testSet() public {
        // Get the deployed contract instance
        SimpleStorage simpleStorage = SimpleStorage(DeployedAddresses.SimpleStorage());

        // Set a value
        uint expectedValue = 42;
        simpleStorage.set(expectedValue);

        // Retrieve the value
        uint storedValue = simpleStorage.get();

        // Assert that the retrieved value matches the expected value
        Assert.equal(storedValue, expectedValue, "Stored value should match the expected value");
    }
}
"""



example['incomplete_example']['contract_code'] = """// SPDX-License-Identifier: MIT
pragma solidity ^0.8.17;

contract Loop {
    function loop() public {
        // for loop
        for (uint i = 0; i < 10; i++) {
            if (i == 3) {
                // Skip to next iteration with continue
                continue;
            }
            if (i == 5) {
                // Exit loop with break
                break;
            }
        }

        // while loop
        uint j;
        while (j < 10) {
            j++;
        }
    }
}
"""
example['incomplete_example']['test_contract_code'] = ""
example['incomplete_example']['function'] = "loop"

In [7]:
agent = AgentsLlmApiCaller(example)
result = agent.get_answer_query(**example['incomplete_example'])

Original task prompt:

Here is the smart contract code to be tested: // SPDX-License-Identifier: MIT
pragma solidity ^0.8.17;

contract Loop {
    function loop() public {
        // for loop
        for (uint i = 0; i < 10; i++) {
            if (i == 3) {
                // Skip to next iteration with continue
                continue;
            }
            if (i == 5) {
                // Exit loop with break
                break;
            }
        }

        // while loop
        uint j;
        while (j < 10) {
            j++;
        }
    }
}

Here is the test contract in Solidity: 
Here is the specific function you need to test in the new end-to-end test: loop
Recommend a new end-to-end test wrapped in a Solidity function with no other comments or explanation.

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol"; 

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    functio

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

...

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

couldn't compile pragma solidity ^0.8.17;

import "remix_tests.sol";   

contract LoopTest {
    Loop loopContract;
    
    function beforeAll() public {
        loopContract = new Loop();
    }
    
    function testLoop() public {
        loopContract.loop();
        Assert.equal(true, true, "Should be true");
    }
}

An error occurred during execution
> command: `/Users/uan/anaconda3/envs/agents/bin/solc --help -`
> return code: `0`
> stdout:
solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if none given or
"-" is used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also 

AttributeError: 'dict' object has no attribute 'content'

In [8]:
example['user_task']

'\nHere is the smart contract code to be tested: {contract_code}\nHere is the test contract in Solidity: {test_contract_code}\nHere is the specific function you need to test in the new end-to-end test: {function}\nRecommend a new end-to-end test wrapped in a Solidity function with no other comments or explanation.'